In [1]:
import lzma
import pickle
from pathlib import Path

import dill
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
import rxmc

In [4]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
prior_path = Path("../prior/prior_distribution.pickle")

In [5]:
output_dir = Path("./elm_posterior_propagation/")
output_dir.mkdir(parents=True, exist_ok=True)

## read in ELM posterior samples

In [6]:
results_full = np.load("../calibration/elm_og_cal_result.npz")
results_full["model_chains"].shape

(64, 100000, 14)

In [30]:
def draw_samples(x, low_cut=None, high_cut=None, n_samples=None):
    if x.ndim == 2:
        x = x.reshape((1, *x.shape))
    elif x.ndim != 3:
        raise ValueError("x must have 2 or 3 dimensions")

    chains, steps, n_params = x.shape

    if low_cut is None:
        low_cut = 0
    if high_cut is None:
        high_cut = steps - 1
    samples = x[:, low_cut:high_cut, ...].reshape(
        (chains * (high_cut - low_cut), n_params)
    )
    if n_samples is None:
        return samples
    else:
        return samples[
            np.random.choice(np.arange(chains * (high_cut - low_cut)), n_samples), ...
        ]

In [22]:
final_samples = results_full["model_chains"][:, -1, :]
posterior_samples = draw_samples(results_full["model_chains"], 20000, 40000, 1000)

In [25]:
og_samples = np.concatenate(
    [np.load("/home/beyerk/db/chex_calibration/results/noqe_cal_sample.npy")]
)
mask = np.ones(15, dtype=bool)
mask[6] = False
og_samples = og_samples[:, mask]

In [31]:
og_samples = draw_samples(og_samples, n_samples=1000)

## read in observations and models

In [32]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

In [33]:
with open(model_dir / "elm_nn_model.pkl", "rb") as f:
    elm_nn_model = dill.load(f)

In [34]:
with open(model_dir / "elm_pp_model.pkl", "rb") as f:
    elm_pp_model = dill.load(f)

## propagate ELM prior into constraint observables

In [35]:
yth_nn = []
for entry_id, obs in tqdm(nn_elastic_obs.items()):
    for x in obs:
        yth_nn.append(
            np.array(
                [elm_nn_model.visualizable_model_prediction(x, *p) for p in og_samples]
            )
        )

100%|█████████████████████████████████████████████████████████████████| 18/18 [02:07<00:00,  7.06s/it]


In [36]:
yth_pp = []
for entry_id, obs in tqdm(pp_elastic_obs.items()):
    for x in obs:
        yth_pp.append(
            np.array(
                [elm_pp_model.visualizable_model_prediction(x, *p) for p in og_samples]
            )
        )

100%|█████████████████████████████████████████████████████████████████| 55/55 [05:38<00:00,  6.15s/it]


## write elm posterior predictive distributions to disk

In [37]:
with open(output_dir / "nn_elm_posterior_og_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [38]:
with open(output_dir / "pp_elm_posterior_og_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)